# RNN Sum 100 - Keras

## Imports

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
# seed for reproducable solutions
np.random.seed(42)

In [4]:
# 10000 samples (8000 train, 2000 test), numbers from 0 to 9, sequences of length 30
samples = 10000
seq_length = 30
epochs = 60
batch_size = 50

## Data creation

In [6]:
def create_data(samples, seq_length):
    x_ = np.random.random_integers(low=0, high=9, size=(samples, seq_length))
    # needs one more dim for keras
    x = x_[:, :, np.newaxis]
    y = np.sum(x_, axis=1) 
    y = y[:,np.newaxis]
    y = y >= 100

    return x, y

# get the numbers from boolean labels
def y_to_nb(y):
    return y.astype(int)

In [8]:
x, y = create_data(samples, seq_length)
x_tr, x_test, y_tr, y_test = train_test_split(x, y, test_size=0.2)

/tmp/ipykernel_13876/3591819934.py:2: DeprecationWarning: This function is deprecated. Please call randint(0, 9 + 1) instead
  x_ = np.random.random_integers(low=0, high=9, size=(samples, seq_length))


In [13]:
x_tr.shape

(8000, 30, 1)

## Defining the model

In [17]:
print('Build model...')
model = Sequential()
# 200 lstm cells
model.add(LSTM(200, input_shape=(seq_length,1)))
# sigmoid is important to use it for binary_crossentropy 

# TimeDistributed layer would create output at each step
# but Recurrent layer would have to return_sequence
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Build model...
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 200)               161600    
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 161,801
Trainable params: 161,801
Non-trainable params: 0
_________________________________________________________________
None


## Training the model

In [15]:
hist = model.fit(x_tr, y_tr, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/60
160/160 [==============================] - 5s 24ms/step - loss: 0.0680 - accuracy: 0.9805 - val_loss: 0.0414 - val_accuracy: 0.9870
Epoch 2/60
160/160 [==============================] - 4s 23ms/step - loss: 0.0488 - accuracy: 0.9869 - val_loss: 0.0352 - val_accuracy: 0.9865
Epoch 3/60
142/160 [=========================>....] - ETA: 0s - loss: 0.0342 - accuracy: 0.9873

KeyboardInterrupt: 

## Evaluating the model

In [8]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Loss: %0.4f, Accuracy: %.2f%%" % (scores[0], scores[1]*100))

Loss: 0.0162, Accuracy: 99.45%


# PyTorch Version

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.nn.init as init

## Numpy Data to Torch DataLoader

In [11]:
x_tr = x_tr.astype(np.float32, copy=False)
tensor_x_tr = torch.Tensor(x_tr)

y_tr = y_tr.astype(np.float32, copy=False)
tensor_y_tr = torch.Tensor(y_tr)

x_test = x_test.astype(np.float32, copy=False)
tensor_x_test = torch.Tensor(x_test)

y_test = y_test.astype(np.float32, copy=False)
tensor_y_test = torch.Tensor(y_test)

# create datasets
train_data = torch.utils.data.TensorDataset(tensor_x_tr, tensor_y_tr) 
test_data = torch.utils.data.TensorDataset(tensor_x_test, tensor_y_test)

train_loader = torch.utils.data.DataLoader(
                dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_data, batch_size=x_test.__len__(), shuffle=False)

## Model

In [12]:
class LSTMNN(nn.Module):
    # just the layers and other variables
    # computation is defined in forward method
    def __init__(self):
        super().__init__()
            
        # 200 lstm cells
        self.num_units = 200
        
        # one layer, with 200 cells 
        # data is passed with batch_size as first dim
        # easier to handle in forward pass
        self.lstm = nn.LSTM(1, self.num_units, batch_first = True)
        
        # 200 cells map to one output
        self.output = nn.Linear(self.num_units, 1)
        
    # helper method
    def init_hidden(self, batch_size):
        # initial hidden state is empty -> zeros
        return (Variable(torch.zeros(1, batch_size, self.num_units)),
                Variable(torch.zeros(1, batch_size, self.num_units)))

    # computation is done here
    # backward pass (backprop) is calculated automatically
    def forward(self, x):
        # get batch_size from data
        # evaluation of the model done on complete test batch
        batch_size = x.size()[0]

        # initial hidden state (step = 0)
        hidden_0 = self.init_hidden(batch_size)
        
        # pass input and first hidden state, to lstm
        # defined in __init__
        lstm_out, last = self.lstm(x, hidden_0)
        # lstm_out contains all hidden states (at each time step)
        # should be possible to use a dense layer on all steps
        # last (tuple) contains hidden and cell state of last step
        
        # take hidden state at last timestep 
        # dim order is changed due to batch_first = True
        out = self.output(lstm_out[:,-1,:])
        # or
        #out = self.output(last[0][0,:,:]) 
        # both are the same
        
        out = F.sigmoid(out)
        return out



In [13]:
model = LSTMNN()
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-8)

In [14]:
def train(epoch):
    # indicates training
    # useful for e.g. dropout
    # here not necessary but good practice
    model.train()
    for batch_idx, (data, label) in enumerate(train_loader):
        # input has to be a autograd.Variable
        # allows automatic differentiation
        data, label = Variable(data), Variable(label)
        # has to done before each optimizer step 
        # (each backprop)
        # optimizer accumulates the changes
        optimizer.zero_grad()

        output = model(data)
        loss = F.binary_cross_entropy(output, label)
        # calculates the backpropagation started at the loss
        loss.backward()
        
        # updates the parameters of the network
        # by specified optimizer
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
            
def test():
    # indicates testing
    # useful for e.g. dropout
    # here not necessary but good practice
    model.eval()
    test_loss = 0
    correct = 0
    for data, label in test_loader:
        # volatile true deactivates
        # the calculation of gradients in backward pass
        # also deactivates all nodes that depend on data 
        # means whole network doesn't calculate gradients
        data, label = Variable(data, volatile=True), Variable(label)
        
        output = model(data)
        test_loss += F.binary_cross_entropy(output, label, 
                        size_average=False).data[0] # sum up batch loss
        pred = output.data.ge(0.5).float() # to 1.0 if >= 0.5 else 0.0
        correct += pred.eq(label.data.view_as(pred)).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [15]:
print("Number of Epochs:", epochs)
for epoch in range(1, epochs+1):
    train(epoch)
    test()

Number of Epochs: 60
Train Epoch: 1 [0/8000 (0%)]	Loss: 0.710394
Train Epoch: 1 [500/8000 (6%)]	Loss: 0.109925
Train Epoch: 1 [1000/8000 (12%)]	Loss: 0.003429
Train Epoch: 1 [1500/8000 (19%)]	Loss: 0.231383
Train Epoch: 1 [2000/8000 (25%)]	Loss: 0.009938
Train Epoch: 1 [2500/8000 (31%)]	Loss: 0.183802
Train Epoch: 1 [3000/8000 (38%)]	Loss: 0.177125
Train Epoch: 1 [3500/8000 (44%)]	Loss: 0.012175
Train Epoch: 1 [4000/8000 (50%)]	Loss: 0.204910
Train Epoch: 1 [4500/8000 (56%)]	Loss: 0.105060
Train Epoch: 1 [5000/8000 (62%)]	Loss: 0.099537
Train Epoch: 1 [5500/8000 (69%)]	Loss: 0.099801
Train Epoch: 1 [6000/8000 (75%)]	Loss: 0.174862
Train Epoch: 1 [6500/8000 (81%)]	Loss: 0.015048
Train Epoch: 1 [7000/8000 (88%)]	Loss: 0.195471
Train Epoch: 1 [7500/8000 (94%)]	Loss: 0.012824

Test set: Average loss: 0.0581, Accuracy: 1979/2000 (99%)

Train Epoch: 2 [0/8000 (0%)]	Loss: 0.012306
Train Epoch: 2 [500/8000 (6%)]	Loss: 0.014550
Train Epoch: 2 [1000/8000 (12%)]	Loss: 0.097608
Train Epoch: 2 [150


Test set: Average loss: 0.0527, Accuracy: 1979/2000 (99%)

Train Epoch: 11 [0/8000 (0%)]	Loss: 0.095484
Train Epoch: 11 [500/8000 (6%)]	Loss: 0.062220
Train Epoch: 11 [1000/8000 (12%)]	Loss: 0.035056
Train Epoch: 11 [1500/8000 (19%)]	Loss: 0.040849
Train Epoch: 11 [2000/8000 (25%)]	Loss: 0.137797
Train Epoch: 11 [2500/8000 (31%)]	Loss: 0.119528
Train Epoch: 11 [3000/8000 (38%)]	Loss: 0.005539
Train Epoch: 11 [3500/8000 (44%)]	Loss: 0.007124
Train Epoch: 11 [4000/8000 (50%)]	Loss: 0.085504
Train Epoch: 11 [4500/8000 (56%)]	Loss: 0.103717
Train Epoch: 11 [5000/8000 (62%)]	Loss: 0.017811
Train Epoch: 11 [5500/8000 (69%)]	Loss: 0.017212
Train Epoch: 11 [6000/8000 (75%)]	Loss: 0.159510
Train Epoch: 11 [6500/8000 (81%)]	Loss: 0.102285
Train Epoch: 11 [7000/8000 (88%)]	Loss: 0.060289
Train Epoch: 11 [7500/8000 (94%)]	Loss: 0.016150

Test set: Average loss: 0.0380, Accuracy: 1979/2000 (99%)

Train Epoch: 12 [0/8000 (0%)]	Loss: 0.007580
Train Epoch: 12 [500/8000 (6%)]	Loss: 0.117576
Train Epoc

Train Epoch: 20 [6500/8000 (81%)]	Loss: 0.000497
Train Epoch: 20 [7000/8000 (88%)]	Loss: 0.076206
Train Epoch: 20 [7500/8000 (94%)]	Loss: 0.000498

Test set: Average loss: 0.0187, Accuracy: 1982/2000 (99%)

Train Epoch: 21 [0/8000 (0%)]	Loss: 0.057912
Train Epoch: 21 [500/8000 (6%)]	Loss: 0.025210
Train Epoch: 21 [1000/8000 (12%)]	Loss: 0.048594
Train Epoch: 21 [1500/8000 (19%)]	Loss: 0.020346
Train Epoch: 21 [2000/8000 (25%)]	Loss: 0.000067
Train Epoch: 21 [2500/8000 (31%)]	Loss: 0.000380
Train Epoch: 21 [3000/8000 (38%)]	Loss: 0.002571
Train Epoch: 21 [3500/8000 (44%)]	Loss: 0.012676
Train Epoch: 21 [4000/8000 (50%)]	Loss: 0.000590
Train Epoch: 21 [4500/8000 (56%)]	Loss: 0.019656
Train Epoch: 21 [5000/8000 (62%)]	Loss: 0.004258
Train Epoch: 21 [5500/8000 (69%)]	Loss: 0.053142
Train Epoch: 21 [6000/8000 (75%)]	Loss: 0.005787
Train Epoch: 21 [6500/8000 (81%)]	Loss: 0.000511
Train Epoch: 21 [7000/8000 (88%)]	Loss: 0.003405
Train Epoch: 21 [7500/8000 (94%)]	Loss: 0.000789

Test set: Aver

Train Epoch: 30 [5000/8000 (62%)]	Loss: 0.055923
Train Epoch: 30 [5500/8000 (69%)]	Loss: 0.001260
Train Epoch: 30 [6000/8000 (75%)]	Loss: 0.003101
Train Epoch: 30 [6500/8000 (81%)]	Loss: 0.000346
Train Epoch: 30 [7000/8000 (88%)]	Loss: 0.021534
Train Epoch: 30 [7500/8000 (94%)]	Loss: 0.002217

Test set: Average loss: 0.0095, Accuracy: 1993/2000 (100%)

Train Epoch: 31 [0/8000 (0%)]	Loss: 0.019578
Train Epoch: 31 [500/8000 (6%)]	Loss: 0.053066
Train Epoch: 31 [1000/8000 (12%)]	Loss: 0.000340
Train Epoch: 31 [1500/8000 (19%)]	Loss: 0.004538
Train Epoch: 31 [2000/8000 (25%)]	Loss: 0.008996
Train Epoch: 31 [2500/8000 (31%)]	Loss: 0.000816
Train Epoch: 31 [3000/8000 (38%)]	Loss: 0.108974
Train Epoch: 31 [3500/8000 (44%)]	Loss: 0.000322
Train Epoch: 31 [4000/8000 (50%)]	Loss: 0.014705
Train Epoch: 31 [4500/8000 (56%)]	Loss: 0.000551
Train Epoch: 31 [5000/8000 (62%)]	Loss: 0.002847
Train Epoch: 31 [5500/8000 (69%)]	Loss: 0.022247
Train Epoch: 31 [6000/8000 (75%)]	Loss: 0.000351
Train Epoch: 3

Train Epoch: 40 [3500/8000 (44%)]	Loss: 0.004358
Train Epoch: 40 [4000/8000 (50%)]	Loss: 0.000017
Train Epoch: 40 [4500/8000 (56%)]	Loss: 0.000316
Train Epoch: 40 [5000/8000 (62%)]	Loss: 0.002856
Train Epoch: 40 [5500/8000 (69%)]	Loss: 0.038579
Train Epoch: 40 [6000/8000 (75%)]	Loss: 0.000300
Train Epoch: 40 [6500/8000 (81%)]	Loss: 0.011406
Train Epoch: 40 [7000/8000 (88%)]	Loss: 0.016843
Train Epoch: 40 [7500/8000 (94%)]	Loss: 0.000058

Test set: Average loss: 0.0095, Accuracy: 1990/2000 (100%)

Train Epoch: 41 [0/8000 (0%)]	Loss: 0.019879
Train Epoch: 41 [500/8000 (6%)]	Loss: 0.006420
Train Epoch: 41 [1000/8000 (12%)]	Loss: 0.033746
Train Epoch: 41 [1500/8000 (19%)]	Loss: 0.000176
Train Epoch: 41 [2000/8000 (25%)]	Loss: 0.000059
Train Epoch: 41 [2500/8000 (31%)]	Loss: 0.001773
Train Epoch: 41 [3000/8000 (38%)]	Loss: 0.000031
Train Epoch: 41 [3500/8000 (44%)]	Loss: 0.013740
Train Epoch: 41 [4000/8000 (50%)]	Loss: 0.000071
Train Epoch: 41 [4500/8000 (56%)]	Loss: 0.063386
Train Epoch: 4

Train Epoch: 50 [2000/8000 (25%)]	Loss: 0.000206
Train Epoch: 50 [2500/8000 (31%)]	Loss: 0.000492
Train Epoch: 50 [3000/8000 (38%)]	Loss: 0.000458
Train Epoch: 50 [3500/8000 (44%)]	Loss: 0.000017
Train Epoch: 50 [4000/8000 (50%)]	Loss: 0.000349
Train Epoch: 50 [4500/8000 (56%)]	Loss: 0.000014
Train Epoch: 50 [5000/8000 (62%)]	Loss: 0.000836
Train Epoch: 50 [5500/8000 (69%)]	Loss: 0.007506
Train Epoch: 50 [6000/8000 (75%)]	Loss: 0.000055
Train Epoch: 50 [6500/8000 (81%)]	Loss: 0.003017
Train Epoch: 50 [7000/8000 (88%)]	Loss: 0.009955
Train Epoch: 50 [7500/8000 (94%)]	Loss: 0.001710

Test set: Average loss: 0.0093, Accuracy: 1989/2000 (99%)

Train Epoch: 51 [0/8000 (0%)]	Loss: 0.009621
Train Epoch: 51 [500/8000 (6%)]	Loss: 0.000024
Train Epoch: 51 [1000/8000 (12%)]	Loss: 0.002693
Train Epoch: 51 [1500/8000 (19%)]	Loss: 0.042458
Train Epoch: 51 [2000/8000 (25%)]	Loss: 0.000013
Train Epoch: 51 [2500/8000 (31%)]	Loss: 0.000306
Train Epoch: 51 [3000/8000 (38%)]	Loss: 0.000105
Train Epoch: 51

Train Epoch: 60 [500/8000 (6%)]	Loss: 0.000024
Train Epoch: 60 [1000/8000 (12%)]	Loss: 0.000063
Train Epoch: 60 [1500/8000 (19%)]	Loss: 0.000038
Train Epoch: 60 [2000/8000 (25%)]	Loss: 0.000018
Train Epoch: 60 [2500/8000 (31%)]	Loss: 0.026290
Train Epoch: 60 [3000/8000 (38%)]	Loss: 0.000027
Train Epoch: 60 [3500/8000 (44%)]	Loss: 0.000891
Train Epoch: 60 [4000/8000 (50%)]	Loss: 0.002646
Train Epoch: 60 [4500/8000 (56%)]	Loss: 0.000266
Train Epoch: 60 [5000/8000 (62%)]	Loss: 0.008454
Train Epoch: 60 [5500/8000 (69%)]	Loss: 0.015476
Train Epoch: 60 [6000/8000 (75%)]	Loss: 0.000244
Train Epoch: 60 [6500/8000 (81%)]	Loss: 0.008485
Train Epoch: 60 [7000/8000 (88%)]	Loss: 0.006309
Train Epoch: 60 [7500/8000 (94%)]	Loss: 0.000306

Test set: Average loss: 0.0173, Accuracy: 1986/2000 (99%)

